In [2]:
from logicqubit.logic import *
from math import *
import random
import numpy as np
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.6


In [3]:
logicQuBit = LogicQuBit(2)

B = Qubit()
A = Qubit()

# generate entangled state |01>-|10>
A.H()
B.CNOT(A)
B.X()
B.Z()

psi = logicQuBit.getPsi()

gates = Gates()
M = {}
M["ab"] = gates.Z().kron((gates.Z()+gates.X())/sqrt(2))
M["a'b"] = gates.X().kron((gates.Z()+gates.X())/sqrt(2))
M["ab'"] = gates.Z().kron((gates.Z()-gates.X())/sqrt(2))
M["a'b'"] = gates.X().kron((gates.Z()-gates.X())/sqrt(2))
ZZ = gates.X().kron(gates.Z())

values = {}
values["ab"] = (psi.adjoint()*M["ab"]*psi).get()[0]
values["a'b"] = (psi.adjoint()*M["a'b"]*psi).get()[0]
values["ab'"] = (psi.adjoint()*M["ab'"]*psi).get()[0]
values["a'b'"] = (psi.adjoint()*M["a'b'"]*psi).get()[0]

print(logicQuBit.getDictPsi())
print(values)

inequality = values["ab"] + values["a'b"] + values["ab'"] - values["a'b'"]
inequality = sqrt(inequality.real * inequality.real)
print("Inequality: "+str(inequality))

{'00': 0j, '01': (0.7071067811865475+0j), '10': (-0.7071067811865475+0j), '11': 0j}
{'ab': array([-0.70710678+0.j]), "a'b": array([-0.70710678+0.j]), "ab'": array([-0.70710678+0.j]), "a'b'": array([0.70710678+0.j])}
Inequality: 2.8284271247461894


In [89]:
M["ab'"].get()

array([[ 0.70710678, -0.70710678,  0.        , -0.        ],
       [-0.70710678, -0.70710678, -0.        , -0.        ],
       [ 0.        , -0.        , -0.70710678,  0.70710678],
       [-0.        , -0.        ,  0.70710678,  0.70710678]])

In [136]:
#psi.get()

In [130]:
gates = Gates()
#rint(gates.Z(1).kron(gates.H(1)).get()) # ab
#print(gates.X(1).kron(gates.H(1)).get()) # a'b
print(gates.Z(1).kron(gates.U3(1,-pi/2,pi/2,pi)).get()) # ab'
#print(gates.X(1).kron(gates.U3(1,-pi/2,pi/2,pi)).get()) # a'b'

[[ 7.07106781e-01+0.00000000e+00j -7.07106781e-01+8.65956056e-17j
   0.00000000e+00+0.00000000e+00j -0.00000000e+00+0.00000000e+00j]
 [-4.32978028e-17-7.07106781e-01j -1.29893408e-16-7.07106781e-01j
   0.00000000e+00-0.00000000e+00j  0.00000000e+00-0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j -0.00000000e+00+0.00000000e+00j
  -7.07106781e-01+0.00000000e+00j  7.07106781e-01-8.65956056e-17j]
 [ 0.00000000e+00-0.00000000e+00j  0.00000000e+00-0.00000000e+00j
   4.32978028e-17+7.07106781e-01j  1.29893408e-16+7.07106781e-01j]]


In [26]:
logicQuBit = LogicQuBit(1)
x = Qubit()
#x.X()

psi = x.getState().get()
x.H()
x.RY(pi/4)
print(x.getDictPsi())

p = logicQuBit.Measure([x])

Mt = (gates.Z()-gates.X()).get()/sqrt(2)
v = np.dot(np.dot(psi.transpose().conjugate(), Mt), psi)

print(Mt)
print(p,v)

{'0': (0.9238795325112866+0j), '1': (0.3826834323650897+0j)}
[[ 0.70710678 -0.70710678]
 [-0.70710678 -0.70710678]]
[(0.8535533905932735+0j), (0.1464466094067262+0j)] [[0.70710678]]


In [137]:
# |<C>| = |<a.b> + <a'.b> + <a.b'> - <a'.b'>| <= 2
def bell_inequality(A, B):
    direction = {"ab":0, "a'b":0, "ab'":0, "a'b'":0}
    count = {"ab":0, "a'b":0, "ab'":0, "a'b'":0}
    
    for a,b in zip(A,B):
        if a[0]=="a" and b[0]=="b":
            direction["ab"] += a[1]*b[1]
            count["ab"] += 1
        elif a[0]=="a'" and b[0]=="b":
            direction["a'b"] += a[1]*b[1]
            count["a'b"] += 1
        elif a[0]=="a" and b[0]=="b'":
            direction["ab'"] += a[1]*b[1]
            count["ab'"] += 1
        elif a[0]=="a'" and b[0]=="b'":
            direction["a'b'"] += a[1]*b[1]
            count["a'b'"] += 1

    direction["ab"] /= count["ab"]
    direction["a'b"] /= count["a'b"]
    direction["ab'"] /= count["ab'"]
    direction["a'b'"] /= count["a'b'"]
    inequality = direction["ab"] + direction["a'b"] + direction["ab'"] - direction["a'b'"]
    inequality = sqrt(inequality * inequality)
    print(direction)
    return inequality

def measurements(eva=False, attempts=10):
    to_minus_one = lambda x: -1 if x == 0 else x
    measure_A = []
    measure_B = []
  
    for i in range(attempts):
        logicQuBit = LogicQuBit(2)

        B = Qubit()
        A = Qubit()

        # generate entangled state |01>-|10>
        A.H()
        B.CNOT(A)
        B.X()
        B.Z()

        # Alice choose the measurement
        measure_type_A = random.choice(["a", "a'"])
        #if "a" in measure_type_A:
            #A.Z()
        if "a'" in measure_type_A:
            A.X()

        # Alice performs the measurement
        m_A = logicQuBit.Measure_One(A)[0]
        m_A = to_minus_one(m_A)

        # Eva performs the measurement
        if eva:
            measure_type_E = random.choice(["b", "b'"])
            if "b" in measure_type_E:
                B.H()
            elif "b'" in measure_type_E:
                B.U3(-pi/2,pi/2,pi)
            m_E = logicQuBit.Measure_One(B)[0]

        # Bob choose the measurement
        measure_type_B = random.choice(["b", "b'"])
        if "b" in measure_type_B:
            B.H()
        elif "b'" in measure_type_B:
            B.U3(-pi/2,pi/2,pi)

        # Bob performs the measurement
        m_B = logicQuBit.Measure_One(B)[0]
        m_B = to_minus_one(m_B)

        measure_A.append([measure_type_A, m_A])
        measure_B.append([measure_type_B, m_B])

    return measure_A, measure_B

measure_A, measure_B = measurements(eva=False, attempts=500)
bell = bell_inequality(measure_A, measure_B)
print(bell.real)

{'ab': 0.11475409836065574, "a'b": 0.014705882352941176, "ab'": 0.08620689655172414, "a'b'": -0.015873015873015872}
0.23153989313833692


In [138]:
print(measure_A[:10])
print(measure_B[:10])

[["a'", -1], ['a', 1], ['a', -1], ['a', 1], ['a', 1], ['a', -1], ["a'", 1], ["a'", 1], ["a'", -1], ["a'", 1]]
[['b', -1], ["b'", 1], ["b'", -1], ["b'", -1], ['b', 1], ["b'", 1], ["b'", 1], ['b', 1], ['b', -1], ['b', -1]]


In [139]:
# without eva
measure_A, measure_B = measurements(eva=False, attempts=200)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'ab': -0.125, "a'b": 0.0, "ab'": 0.0, "a'b'": -0.07692307692307693}
0.04807692307692307


In [140]:
# without eva
measure_A, measure_B = measurements(eva=True, attempts=200)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'ab': -0.043478260869565216, "a'b": 0.2653061224489796, "ab'": 0.09803921568627451, "a'b'": -0.07407407407407407}
0.393941151339763
